# Metadata to DNA

# Imports

In [ ]:
import numpy as np
import math
import random

# Encoding

# Huffmann encoding
Using Huffman encoding to convert a number to a triplet base system which is then encoded into DNA

In [ ]:
def SegmentAmount2DNA(number) -> str:
    base_3 = np.base_repr(number,base=3)
    meta_string=''

    A_dic = {'0':"C", '1': "G", '2':"T"}
    C_dic = {'0':"G", '1': "T", '2':"A"}
    G_dic = {'0':"T", '1': "A", '2':"C"}
    T_dic = {'0':"A", '1': "C", '2':"G"}

    for i in range(len(base_3)):
        if i == 0:
            nucleotide = A_dic[base_3[i]]
        elif meta_string[i-1] == 'A':
            nucleotide = A_dic[base_3[i]]
        elif meta_string[i-1] == 'C':
            nucleotide = C_dic[base_3[i]]
        elif meta_string[i-1] == 'G':
            nucleotide = G_dic[base_3[i]]
        elif meta_string[i-1] == 'T':
            nucleotide = T_dic[base_3[i]]
        meta_string = meta_string + nucleotide
    return meta_string

# Create random segment

In [ ]:
def random_segment(meta_string):
    random_segment = ''
    j = int(math.ceil(((50 - len(meta_string))/2)))
    for i in range(j):
        x = random.randint(1,4)
        if x == 1:
            random_segment += 'A'
        elif x == 2:
            random_segment += 'T'
        elif x == 3:
            random_segment += 'C'
        elif x == 4:
            random_segment += 'G'
    reverse = random_segment[::-1]
    if (j % 2) == 0:
        random_segment += reverse
    else:
        reverse = reverse[1:]
        random_segment += reverse
    print(random_segment)
    return random_segment

# Biochemical requirements

In [ ]:
def CheckBiochemicalRequirements(s, max_length=3, check=False):
    if s.find((max_length+1)*'C') ==-1 & s.find((max_length+1)*'G') ==-1 & s.find((max_length+1)*'T')==-1 & s.find((max_length+1)*'A')==-1:
 #       print('yes, no homopolymers')
        nr_CG = s.count('C')+s.count('G')
        per_CG=nr_CG/len(s)
 #       print(per_CG)
        if (per_CG < 0.55) & (per_CG > 0.45):
 #           print('Suitable CG content')
            check = True
      #  else:
 #           print('CG content not in accepted ranges')
    #else:
 #        print('not a valid sequence: it contains homopolymers longer than three bases')
    return check

# Main script metasegment encode

In [ ]:
def metasegment(number)->str:
    meta_string = SegmentAmount2DNA(number)
    metasegment = ''
    while len(metasegment) < 1:
        Random = random_segment(meta_string)
        metadata = meta_string + Random
        if CheckBiochemicalRequirements(metadata) == True:
            metasegment += metadata
    metasegment += metasegment
    return metasegment

# Decoding

# Huffman decoding

In [ ]:
def DNA2SegmentLength(s):
    base_3 = ''
    A_dic = {'C':"0", 'G': "1", 'T':"2"}
    C_dic = {'G':"0", 'T': "1", 'A':"2"}
    G_dic = {'T':"0", 'A': "1", 'C':"2"}
    T_dic = {'A':"0", 'C': "1", 'G':"2"}

    for i in range(len(s)):
        if i == 0:
            digit = A_dic[s[i]]
        elif s[i-1] == 'A':
            digit = A_dic[s[i]]
        elif s[i-1] == 'C':
            digit = C_dic[s[i]]
        elif s[i-1] == 'G':
            digit = G_dic[s[i]]
        elif s[i-1] == 'T':
            digit = T_dic[s[i]]
        base_3 = base_3 + digit
    segment_length=0
    for i in range(-1,-len(base_3)-1,-1):
        segment_length = segment_length + int(base_3[i])*3**(-i-1)
    
    return segment_length